# Building your own TensorFlow - SCANN container

With Amazon SageMaker, you can package your own algorithms that can then be trained and deployed in the SageMaker environment. This notebook guides you through an example using TensorFlow that shows you how to build a Docker container for SageMaker and use it for real time inference.

## Permissions

Running this notebook requires permissions in addition to the normal `SageMakerFullAccess` permissions. This is because it creates new repositories on Amazon ECR. The easiest way to add these permissions is simply to add the managed policy `AmazonEC2ContainerRegistryFullAccess` to the role that you used to start your notebook instance. There's no need to restart your notebook instance when you do this, the new permissions will be available immediately.

## The example

In this example we show how to package a custom TensorFlow container with a trained SCANN index on the movie lens dataset.  

### The parts of the sample container

The `container` directory has all the components you need to package the sample algorithm for `Amazon SageMager`:
```
    .
    |-- Dockerfile
    |-- build_and_push.sh
    |-- savedscann
    |   |-- 1
    |       |-- assets
    |       |-- variables
    |       |-- saved_model.pb
    |-- code
        |-- nginx.conf
        |-- serve

```

Let's discuss each of these in turn:

* __`Dockerfile`__ describes how to build your Docker container image. More details are provided below.
* __`build_and_push.sh`__ is a script that uses the `Dockerfile` to build your container images and then pushes it to ECR. We invoke the commands directly later in this notebook, but you can just copy and run the script for your own algorithms.
* __`code`__ is the directory which contains the files that are installed in the container.
* __`scavedscann`__ is the directory which contains the files that are the uncompressed model artifacts

### The `Dockerfile`

The `Dockerfile` describes the image that we want to build. You can think of it as describing the complete operating system installation of the system that you want to run. A Docker container running is quite a bit lighter than a full operating system, however, because it takes advantage of Linux on the host machine for the basic operations.

For the Python science stack, we start from an official TensorFlow serving scann docker image. Then we add the code that implements our specific algorithm to the container and set up the right environment for it to run under.

Let's look at the `Dockerfile` for this example.

In [43]:
!cat Dockerfile

# SageMaker TF Inference image
FROM google/tf-serving-scann

RUN apt-get -y update && apt-get install -y --no-install-recommends \
         wget \
         python3-pip \
         python3-setuptools \
         nginx \
         ca-certificates

RUN ln -s /usr/bin/python3 /usr/bin/python
RUN ln -s /usr/bin/pip3 /usr/bin/pip
ENV PATH="/opt/program:${PATH}"

# Set up the program in the image
COPY code /opt/ml/code
COPY savedscann /opt/ml/model/index
WORKDIR /opt/ml/code
ENTRYPOINT ["python","serve"]


### Building and registering the container

The following shell code shows how to build the container image using `docker build` and push the container image to ECR using `docker push`. This code is also available as the shell script `build-and-push.sh`, which you can run as `build-and-push.sh` to build the image `sagemaker-tf-scann-example`. 

This code looks for an ECR repository in the account you're using and the current default region (if you're using a SageMaker notebook instance, this is the region where the notebook instance was created). If the repository doesn't exist, the script will create it.

In [49]:
!cat build_and_push.sh

%%sh

# The name of our algorithm
algorithm_name=sagemaker-tf-scann-example

chmod +x code/train
chmod +x code/serve

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-us-west-2}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.

aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
$(aws ecr get-login --region ${region} --no-include-email)

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build  -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

## Hosting your Algorithm in Amazon SageMaker
Once you have your container packaged, you can use it to serve models. Let's do that with the algorithm we made above.


In [29]:
from sagemaker.model import Model
from sagemaker.estimator import Estimator
from sagemaker import get_execution_role

In [56]:
role = get_execution_role()
ecr_image = '431615879134.dkr.ecr.us-east-1.amazonaws.com/sagemaker-tf-scann-example:latest'
instance_type = "ml.m4.xlarge"

scann_model = Model(
    role=role,
    image_uri=ecr_image,
)


predictor = scann_model.deploy(1, instance_type)

-----!

In [57]:
scann_model.endpoint_name

'sagemaker-tf-scann-example-2022-10-31-20-01-30-897'

In [60]:
!aws sagemaker-runtime invoke-endpoint \
--endpoint-name sagemaker-tf-scann-example-2022-10-31-20-01-30-897 \
--body '{"instances": ["42","43"]}' prediction_response.json

{
    "ContentType": "application/json",
    "InvokedProductionVariant": "AllTraffic"
}


In [61]:
!cat prediction_response.json

{
    "predictions": [
        {
            "output_1": [2.19959259, 2.11500525, 1.87805462, 1.87397075, 1.82147372, 1.81811309, 1.77092397, 1.76034117, 1.75835824, 1.7510314],
            "output_2": ["Kid in King Arthur's Court, A (1995)", "Homeward Bound: The Incredible Journey (1993)", "Cool Runnings (1993)", "Angels in the Outfield (1994)", "Bridges of Madison County, The (1995)", "Aristocats, The (1970)", "Winnie the Pooh and the Blustery Day (1968)", "Lion King, The (1994)", "Jungle Book, The (1994)", "D3: The Mighty Ducks (1996)"]
        },
        {
            "output_1": [1.78076386, 1.69247246, 1.68209136, 1.65398324, 1.64403212, 1.59631383, 1.49829984, 1.49201, 1.47684073, 1.45613444],
            "output_2": ["Now and Then (1995)", "Cool Runnings (1993)", "Only You (1994)", "Little Women (1994)", "Corrina, Corrina (1994)", "Fox and the Hound, The (1981)", "Affair to Remember, An (1957)", "Walk in the Clouds, A (1995)", "While You Were Sleeping (1995)", "Homeward Bound: 

## Optional cleanup
When you're done with the endpoint, you should clean it up.

All the training jobs, models and endpoints we created can be viewed through the SageMaker console of your AWS account.

In [ ]:
predictor.delete_endpoint()

## Reference
- [How Amazon SageMaker interacts with your Docker container for training](https://docs.aws.amazon.com/sagemaker/latest/dg/your-algorithms-training-algo.html)
- [How Amazon SageMaker interacts with your Docker container for inference](https://docs.aws.amazon.com/sagemaker/latest/dg/your-algorithms-inference-code.html)
- [CIFAR-10 Dataset](https://www.cs.toronto.edu/~kriz/cifar.html)
- [SageMaker Python SDK](https://github.com/aws/sagemaker-python-sdk)
- [`Dockerfile`](https://docs.docker.com/engine/reference/builder/)
- [scikit-bring-your-own](https://github.com/awslabs/amazon-sagemaker-examples/blob/master/advanced_functionality/scikit_bring_your_own/scikit_bring_your_own.ipynb)